# Face Mask Detector ( Using Transfer Learning)


**Objective**
* To train a custom deep learning model to detect whether a person is or is not wearing a mask using the COVID-19 face mask detection dataset( I will discuss about it in detail ahead, I promise, till then bare with me...👍)


**Implemented in two phases**
<img src="utils/face_mask_detector_phases_1.jpg">

`In order to train a custom face mask detector, we need to break our project into two distinct phases, each with its own respective sub-steps(as shown by the image above 👆:`

**Training:** This part mostly focusses on loading our face mask detection dataset from disk, training a model (using Keras/TensorFlow) on the `COVID-19 face mask detection dataset`.

**Deployment:** Once the face mask detector is trained, we can then move on to loading the mask detector, performing face detection, and then classifying each face as `with_mask` or `without_mask`.
    









## 1. Required Imports 💼

These are the libraries and their functions which are required for the execution of the project and the commonly used libararies are **os, pahtllib** (both are mainly used for fetching the data from the local files), **matplotlib** (for visualizing purposes), **numpy, random, tensorflow, sklearn** (related to the creation of the model, processing and analysing the data).

In [ ]:
import os
import pathlib
import imutils
from imutils import paths

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import time
import cv2
from imutils.video import VideoStream # For starting the videostream where the model applies it objectives.

import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


## 2. Data Exploration and Visualization Step

**Walk through the directories and list number of file**

To check the folders and its contents

In [ ]:
# Set up the dir path
dir_path = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset"

for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}.")

**To get the class names programatically** 

`Optional`, can be done manually also but I did it this way becuase I prefer doing it programmatically😅.

In [ ]:
# Get the class names programmatically
dir_path_train = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\train"

data_dir = pathlib.Path(dir_path_train)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
print(class_names)

**Visualizing few images** 

Data Exploers motto is `Visualize Visualize Visualize...`

In [ ]:
def view_random_image(target_dir, target_class):
    """
    This function picks up a random image image from the target
    directory and visualizes it and also prints the shape of it.
    """
    # Setup the target directory
    target_folder = target_dir+target_class

    # Get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

    # Read in the image and plot it using matplotlib
    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off")

    print(f"Image Shape: {img.shape}") # Show the shape of the image

    return img

In [ ]:
# Let's visualize an image with mask
train_dir = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\train\\"

img = view_random_image(target_dir=train_dir,
                        target_class=class_names[0]);

In [ ]:
# And now one without mask (Just for the sake of the data explorers motto)
img = view_random_image(target_dir=train_dir,
                        target_class=class_names[1]);

## 2. Data Preprocessing

**Let's discuss about the data (as promised)**

`About the data:`

* This dataset consists of `1650` images belonging to two classes: 1. `with_mask` & `without_mask`.


`To create the dataset:`

* Taking normal images of faces
* Then creating a custom computer vision Python script to add face masks to them, thereby creating an artificial (but still real-world applicable) dataset.

This method is actually a lot easier than it sounds once you apply facial landmarks to the problem.

Facial landmarks allow us to automatically infer the location of facial structures, including:

* Eyes
* Eyebrows
* Nose
* Mouth
* Jawline

`Steps to apply it:`
* To use facial landmarks to build a dataset of faces wearing face masks, we need to first start with an image of a person not wearing a face mask.

* From there, we apply face detection to compute the bounding box location of the face in the image.

* Once we know where in the image the face is, we can extract the face Region of Interest (ROI).

* And from there, we apply facial landmarks, allowing us to localize the eyes, nose, mouth, etc.

* Next, we need an image of a mask (with a transparent background).

* This mask will be automatically applied to the face by using the facial landmarks (namely the points along the chin and nose) to compute where the mask will be placed.

* The mask is then resized and rotated, placing it on the face

* We can then repeat this process for all of our input images, thereby creating our artificial face mask dataset

However, there is a something you should be aware of when using this method to artificially create a dataset!

If you use a set of images to create an artificial dataset of people wearing masks, you cannot “re-use” the images without masks in your training set — you still need to gather non-face mask images that were not used in the artificial generation process!

If you include the original images used to generate face mask samples as non-face mask samples, your model will become heavily biased and fail to generalize well. Avoid that at all costs by taking the time to gather new examples of faces without masks.

This was all about the dataset i hope you found it useful.

Now, let's understant the preprocessing part...

Firstly, we will scale the data between 0 and 1 and aggregate them into batches of size 32 and will resize the image to a target size of (224, 224) height & width.

Now, What is ImageDataGenerator? 🤔

**ImageDataGenerator** class allows the users to perform image transformation while training the model and is helpful when there are large datasets which can't be loaded and transformed at once due to limited system resources.

For more refer to this: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator 




In [ ]:
# Set the seed (You can think of it as a number which describes a one type of arrangement of data and it changes for every Number)
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 0 & 1, also called scaling/Normalizing)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_dataget = ImageDataGenerator(rescale=1./255)

# Setup paths to our directories
train_dir = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\train"
valid_dir = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\val"
test_dir =  r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\test"

# Import the data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(directory=train_dir,
                                            batch_size=32,
                                            target_size=(224, 224),
                                            class_mode="categorical",
                                            seed=42)

valid_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

test_data = valid_datagen.flow_from_directory(directory=test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

## 3. Create, Compile & Train the model

To accomplish this task, we’ll be using Transfer Learning and will perfomr fine-tuning on the MobileNet V2 architecture, a highly efficient architecture that can be applied to embedded devices with limited computational capacity (ex., Raspberry Pi, Google Coral, NVIDIA Jetson Nano, etc.)


`Something about MobileNetV2 model:`
MobileNetV2 is a convolutional neural network architecture that seeks to perform well on mobile devices. It is based on an inverted residual structure where the residual connections are between the bottleneck layers. The intermediate expansion layer uses lightweight depthwise convolutions to filter features as a source of non-linearity. As a whole, the architecture of MobileNetV2 contains the initial fully convolution layer with 32 filters, followed by 19 residual bottleneck layers.

<img src="utils/mobillenetv2.jpg">
For better understanding refer to this:

https://arxiv.org/abs/1801.04381v4

In [ ]:
## Required Variables
INIT_LR = 1e-4 # This has been achieved after performing lots of experiments and not a random number.
EPOCHS = 20

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# We place the head model on top of the base model (this will become the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# Looping over all layers in the base model and freeze them so they will *not* be updated during the first training process and no weights will be lost
for layer in baseModel.layers:
    layer.trainable = False
    
# Compiling our model
print("Compiling the model")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

In [ ]:
# To get the summary and structure of the model
model.summary()

In [ ]:
# Training our model
history = model.fit(train_data,
                   epochs=EPOCHS,
                   steps_per_epoch=len(train_data),
                   validation_data = valid_data,
                   validation_steps=len(valid_data))

## 4. Evaluating the model

In [ ]:
def path_loss_curves(history):
    """
    Returns separate loss curves for training and validation dataset.
    """
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    accuracy = history.history["accuracy"]
    val_accuracy = history.history["val_accuracy"]

    epochs = range(len(history.history["loss"])) # How many times did we run for?

    # Plot the loss
    plt.plot(epochs, loss, label="loss")
    plt.plot(epochs, val_loss, label="val_loss")
    plt.title("loss")
    plt.xlabel("epochs")
    plt.legend()

    # Plot the accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label="accuracy")
    plt.plot(epochs, val_accuracy, label="val_accuracy")
    plt.title("accuracy")
    plt.xlabel("epochs")
    plt.legend();


In [ ]:
# Plot the training and validation curves separately
path_loss_curves(history)

## 5. Making Predictions using our trained model on a test dataset image 

In [ ]:
def load_and_prep_image(filename, img_size=224):
    """
    Reads an image from a file name, turns into a tensor and
    resizes it to [img_size, img_size, colour_channel]
    """

    # Read the image from file
    img = tf.io.read_file(filename)
    # Decode the read image into tensor
    img = tf.image.decode_image(img)
    # Resize the image
    img = tf.image.resize(img, size=[img_size, img_size])
    # Rescale the image (Turn the values between 0 and 1)
    img = img/255.

    return img

In [ ]:
# Create a function to import, preprocess, predict and plot
def pred_and_plot(model, filename, class_names=class_names):
    """
    Imports the image present at filename, makes a prediction on it
    and plots image with its predicted label
    """

    # Read the image from file name
    img = load_and_prep_image(filename)

    # Make prediction on the images
    pred = model.predict(tf.expand_dims(img, axis=0))

    # Get the prediction class
    class_pred = class_names[np.argmax(pred)]

    # Plot the image and its predicted class
    plt.imshow(img)
    plt.title(f"Prediction: {class_pred}")
    plt.axis(False);

In [ ]:
# Checkiing the class names
class_names

In [ ]:
# Prediction with with_mask image
filename = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\test\with_mask\1-with-mask.jpg"
pred_and_plot(model=model,
             filename=filename,
             class_names=class_names)

In [ ]:
# Prediction with without_mask image
filename = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\dataset\test\without_mask\11.jpg"
pred_and_plot(model=model,
             filename=filename,
             class_names=class_names)

## 6.Saving the model

In [ ]:
print("Saving mask detector model...")
model_path = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\Model\\mask_detection.model"
model.save(model_path, save_format="h5")

## 7. Code to start the live video stream and real-time face mask detection

**What is .caffemodel extension?**

A CAFFEMODEL file is a machine learning model created by Caffe. It contains an image classification or image segmentation model that has been trained using Caffe. CAFFEMODEL files are created from .PROTOTXT files.


**What is .protottxt extension?**

A PROTOTXT file is a prototype machine learning model created for use with Caffe. It contains an image classification or image segmentation model that is intended to be trained in Caffe. PROTOTXT files are used to create .CAFFEMODEL files.


`res10_300x300_ssd_iter_140000.caffemodel`uses a ResNet-10 neural network is an architecture/model to detect the faces and their background, i.e. Deep learning-based face detection (Caffe/TensorFlow architecture).


In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
        (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            
            
            
            
            

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
            
            

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding
    # locations
    return (locs, preds)


In [ ]:
# loading our serialized face detector model from disk
prototxtPath = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\facedetector\deploy.prototxt"
weightsPath = r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\facedetector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# loading the face mask detector model from disk
maskNet = load_model(r"D:\MachineLearningProjects\project7sem\DIC_FINAL_PROJECT\FACE_MASK_DETECTION\Model\mask_detection.model")

# Initialize the video stream
print("Starting Video Stream...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
    # Grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # Detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # determine the class label and color we'll use to draw
        # the bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # To display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # If key 'q' is pressed then end the stream and exit from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()



## 8. Few outputs of model

**The output of model when a person has worn a mask**
<img src="utils/With_mask_prediction.jpg">

**The output of model when a person hasn't worn a mask**
<img src="utils/Without_mask_prediction.jpg">